<a href="https://colab.research.google.com/github/Abdullah-Al-Arafat/CAP-5610-Machine-Learning/blob/master/ML_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CAP 5610: Machine Learning** 
Home work 2


In [1]:
import os
import numpy as np
from keras.datasets import cifar10
from keras import Sequential
from keras.utils import np_utils
from keras.optimizers import Adagrad
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten, BatchNormalization 

Using TensorFlow backend.


Preparation/preprocessing of train, test, and validation dataset 

In [0]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

## Creating validation set from training dataset

indices = np.random.permutation(len(train_images))

val_indices = indices[0:2000]
train_indices = indices[2000:]

val_images, val_labels = train_images[val_indices], train_labels[val_indices]

Train_images, Train_labels = train_images[train_indices], train_labels[train_indices]


## Normalizing the Train, test and validation sets

X_train = (Train_images/255).astype('float32')
y_train = np_utils.to_categorical(Train_labels, 10)

X_val = (val_images/255).astype('float32')
y_val = np_utils.to_categorical(val_labels, 10)

X_test = (test_images/255).astype('float32')
y_test = np_utils.to_categorical(test_labels, 10)

In [3]:

## Build the CNN model with keras

model = Sequential()
model.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
 
model.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 48)        13872     
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 48)        192       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 48)          0         
_________________________________________________________________
conv

In [0]:
# Initiate Adagrad/RMSprop optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [5]:
## Train the model and get the training and validation loss
history = model.fit(X_train, y_train, batch_size= 40, epochs= 30,
                    verbose=1, validation_data=(X_val, y_val))


Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 2000 samples
Epoch 1/30
48000/48000 [==============================] - 22s 461us/step - loss: 1.3151 - acc: 0.5337 - val_loss: 1.1191 - val_acc: 0.6135
Epoch 2/30
48000/48000 [==============================] - 20s 412us/step - loss: 0.9367 - acc: 0.6689 - val_loss: 1.0319 - val_acc: 0.6405
Epoch 3/30
48000/48000 [==============================] - 20s 412us/step - loss: 0.7853 - acc: 0.7269 - val_loss: 1.2315 - val_acc: 0.5815
Epoch 4/30
48000/48000 [==============================] - 20s 408us/step - loss: 0.6821 - acc: 0.7600 - val_loss: 1.0347 - val_acc: 0.6610
Epoch 5/30
48000/48000 [==============================] - 19s 404us/step - loss: 0.6027 - acc: 0.7881 - val_loss: 1.0101 - val_acc: 0.6685
Epoch 6/30
48000/48000 [==============================] - 20s 420us/step - loss: 0.5397 - acc: 0.8108 - val_loss: 0.8770 - val_acc: 0.7155
Epoch 7/30
48000/48000 [==============================] - 19s 404us/s

In [6]:
## Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 137us/step
Test loss: 1.4097728177070619
Test accuracy: 0.7249
